<a href="https://colab.research.google.com/github/klotardesignstudio/tradebao-cripto-poc/blob/main/04_backtest_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TRADEBAO POC - NOTEBOOK 04: BACKTEST DE PORTFÓLIO (V2 - ANÁLISE DE TRADES)

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# 1. IMPORTS E FUNÇÃO RSI

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def calculate_rsi(data, period=14):
    close_delta = data['Close'].diff()
    gain = close_delta.clip(lower=0)
    loss = -1 * close_delta.clip(upper=0)
    avg_gain = gain.ewm(com=period - 1, min_periods=period).mean()
    avg_loss = loss.ewm(com=period - 1, min_periods=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    data['RSI'] = rsi
    return data

In [25]:
# 2. NOVO MOTOR DE BACKTEST UNIFICADO E DETALHADO


def backtest_engine(df_original, capital_inicial, strategy_type, taxa_transacao=0.001):
    """
    Motor de backtest genérico que agora retorna o capital final E um log detalhado de trades.
    """
    df = df_original.copy()

    # --- Lógica de Sinais (depende da estratégia) ---
    if strategy_type == 'Tendencia':
        ema_curta, ema_longa, sma_tendencia = 21, 55, 100
        df['EMA_curta'] = df['Close'].ewm(span=ema_curta, adjust=False).mean()
        df['EMA_longa'] = df['Close'].ewm(span=ema_longa, adjust=False).mean()
        df['SMA_tendencia'] = df['Close'].rolling(window=sma_tendencia).mean()
        df.dropna(inplace=True)
        filtro_tendencia = df['Close'] > df['SMA_tendencia']
        df['sinal_compra'] = (df['EMA_curta'] > df['EMA_longa']) & (df['EMA_curta'].shift(1) < df['EMA_longa'].shift(1)) & filtro_tendencia
        df['sinal_venda'] = (df['EMA_curta'] < df['EMA_longa']) & (df['EMA_curta'].shift(1) > df['EMA_longa'].shift(1))

    elif strategy_type == 'RSI':
        rsi_periodo, rsi_sobrevenda, rsi_saida = 14, 30, 50
        df = calculate_rsi(df, period=rsi_periodo)
        df.dropna(inplace=True)
        df['sinal_compra'] = (df['RSI'].shift(1) > rsi_sobrevenda) & (df['RSI'] <= rsi_sobrevenda)
        df['sinal_venda'] = (df['RSI'].shift(1) < rsi_saida) & (df['RSI'] >= rsi_saida)

    # --- Loop de Backtest com Log Detalhado ---
    capital_btc = capital_inicial
    em_posicao = False
    trades_log = []
    entry_data = {}

    # O capital só é alterado pelos resultados dos trades, vamos calculá-lo no final
    # A variável 'capital_btc' agora representa o crescimento percentual
    crescimento_acumulado = 1.0

    for i, row in df.iterrows():
        if row['sinal_compra'] and not em_posicao:
            em_posicao = True
            entry_data = {'data_entrada': i, 'preco_entrada': row['Close']}

        elif row['sinal_venda'] and em_posicao:
            em_posicao = False

            # Calcula o resultado do trade
            resultado_pct = ((row['Close'] - entry_data['preco_entrada']) / entry_data['preco_entrada'])
            resultado_liquido_pct = resultado_pct - (2 * taxa_transacao) # Deduz taxa de entrada e saída
            crescimento_acumulado *= (1 + resultado_liquido_pct)

            # Log do trade
            duracao = i - entry_data['data_entrada']
            trades_log.append({
                **entry_data,
                'data_saida': i,
                'preco_saida': row['Close'],
                'duracao': duracao,
                'resultado_bruto_pct': resultado_pct * 100,
                'resultado_liquido_pct': resultado_liquido_pct * 100
            })
            entry_data = {}

    capital_final = capital_inicial * crescimento_acumulado

    # Se terminar em posição, liquidamos no último preço para o cálculo do capital final
    if em_posicao:
        last_price = df['Close'].iloc[-1]
        resultado_pct = ((last_price - entry_data['preco_entrada']) / entry_data['preco_entrada'])
        resultado_liquido_pct = resultado_pct - taxa_transacao # Só taxa de saída
        capital_final = (capital_inicial / entry_data['preco_entrada']) * last_price * (1 - taxa_transacao)


    return capital_final, pd.DataFrame(trades_log)

In [26]:
# 3. CONFIGURAÇÕES E EXECUÇÃO DO PORTFÓLIO

# Começando com uma lista menor para uma análise de comportamento mais rápida
simbolos = [
    # Categoria: Inteligência Artificial (AI)
    "fetbtc", "rndrbtc", "agixbtc", "oceanbtc", "taobtc", "grtbtc", "phabtc", "rlcbtc",

    # Categoria: Memecoins & High-Beta L1s
    "dogebtc", "zilbtc", "icxbtc", "nearbtc", "pepeusdt", "wifusdt", "seibtc", "aptbtc",

    # Categoria: Infraestrutura (L1/L2)
    "solbtc", "avaxbtc", "ethbtc", "opbtc", "injbtc", "maticbtc", "sushibtc", "adabtc",

    # Categoria: Gaming / Metaverso
    "imxbtc", "flowbtc", "galabtc", "axsbtc", "manabtc", "sandbtc", "apebtc", "enjbtc",

    # Categoria: DePIN & RWA
    "iotxbtc", "filbtc", "arbtc", "thetabtc", "icpbtc", "ombtc", "aavebtc", "pendlebtc",
]
capital_total_inicial = 1.0
alocacao_tendencia = 0.5
alocacao_rsi = 0.5

# Listas para coletar os logs de todos os trades de todos os ativos
log_total_tendencia = []
log_total_rsi = []
# Lista para o resumo de rentabilidade (como antes)
resultados_portfolio = []

for simbolo in simbolos:
    # IMPORTANTE: Garanta que o nome do arquivo corresponde ao timeframe que você quer analisar (ex: _1h.csv)
    caminho_do_arquivo = f'/content/drive/MyDrive/Colab Notebooks/TradeBaoCripto/dados_binance_{simbolo}_15m.csv'

    try:
        df_ativo = pd.read_csv(caminho_do_arquivo, index_col='Open Time', parse_dates=True)
        print(f"----- Processando {simbolo.upper()} em 1h -----")
    except FileNotFoundError:
        print(f"ERRO: Arquivo não encontrado para {simbolo}. Pulando.")
        continue

    # Rodar os backtests e coletar os logs
    resultado_final_tendencia, trades_tendencia = backtest_engine(df_ativo, capital_total_inicial * alocacao_tendencia, 'Tendencia')
    resultado_final_rsi, trades_rsi = backtest_engine(df_ativo, capital_total_inicial * alocacao_rsi, 'RSI')

    # Adiciona os logs de trade às listas totais
    if not trades_tendencia.empty:
        log_total_tendencia.append(trades_tendencia)
    if not trades_rsi.empty:
        log_total_rsi.append(trades_rsi)

    # O resto do cálculo de rentabilidade continua como antes
    capital_final_portfolio = resultado_final_tendencia + resultado_final_rsi
    rent_tendencia = ((resultado_final_tendencia - (capital_total_inicial * alocacao_tendencia)) / (capital_total_inicial * alocacao_tendencia)) * 100
    rent_rsi = ((resultado_final_rsi - (capital_total_inicial * alocacao_rsi)) / (capital_total_inicial * alocacao_rsi)) * 100
    rent_portfolio = ((capital_final_portfolio - capital_total_inicial) / capital_total_inicial) * 100
    rent_bh = ((df_ativo['Close'].iloc[-1] - df_ativo['Close'].iloc[0]) / df_ativo['Close'].iloc[0]) * 100 if not df_ativo.empty else 0
    resultados_portfolio.append({'Ativo': simbolo, 'Rent. Tendência (%)': rent_tendencia, 'Rent. RSI (%)': rent_rsi, 'Rent. PORTFÓLIO (%)': rent_portfolio, 'Rent. B&H (%)': rent_bh})

----- Processando FETBTC em 1h -----
----- Processando RNDRBTC em 1h -----
----- Processando AGIXBTC em 1h -----
----- Processando OCEANBTC em 1h -----
----- Processando TAOBTC em 1h -----
----- Processando GRTBTC em 1h -----
----- Processando PHABTC em 1h -----
----- Processando RLCBTC em 1h -----
----- Processando DOGEBTC em 1h -----
----- Processando ZILBTC em 1h -----
----- Processando ICXBTC em 1h -----
----- Processando NEARBTC em 1h -----
----- Processando PEPEUSDT em 1h -----
----- Processando WIFUSDT em 1h -----
----- Processando SEIBTC em 1h -----
----- Processando APTBTC em 1h -----
----- Processando SOLBTC em 1h -----
----- Processando AVAXBTC em 1h -----
----- Processando ETHBTC em 1h -----
----- Processando OPBTC em 1h -----
----- Processando INJBTC em 1h -----
----- Processando MATICBTC em 1h -----
----- Processando SUSHIBTC em 1h -----
----- Processando ADABTC em 1h -----
----- Processando IMXBTC em 1h -----
----- Processando FLOWBTC em 1h -----
----- Processando GALABT

In [27]:
# 4. NOVA SEÇÃO: ANÁLISE DE COMPORTAMENTO DAS ESTRATÉGIAS

# Se a lista de logs não estiver vazia, concatena tudo em um único DataFrame e analisa
if log_total_tendencia:
    df_tendencia_trades = pd.concat(log_total_tendencia)
    print("\n\n===== ANÁLISE DE COMPORTAMENTO - ESTRATÉGIA DE TENDÊNCIA =====")
    print(f"Número Total de Trades: {len(df_tendencia_trades)}")
    print(f"Duração Média de um Trade: {df_tendencia_trades['duracao'].mean()}")
    print(f"Duração Máxima de um Trade: {df_tendencia_trades['duracao'].max()}")
    print(f"Taxa de Acerto (Win Rate): {((df_tendencia_trades['resultado_liquido_pct'] > 0).sum() / len(df_tendencia_trades) * 100):.2f}%")
    print(f"Lucro Médio (nos trades vencedores): {df_tendencia_trades[df_tendencia_trades['resultado_liquido_pct'] > 0]['resultado_liquido_pct'].mean():.2f}%")
    print(f"Prejuízo Médio (nos trades perdedores): {df_tendencia_trades[df_tendencia_trades['resultado_liquido_pct'] < 0]['resultado_liquido_pct'].mean():.2f}%")

if log_total_rsi:
    df_rsi_trades = pd.concat(log_total_rsi)
    print("\n\n===== ANÁLISE DE COMPORTAMENTO - ESTRATÉGIA RSI =====")
    print(f"Número Total de Trades: {len(df_rsi_trades)}")
    print(f"Duração Média de um Trade: {df_rsi_trades['duracao'].mean()}")
    print(f"Duração Máxima de um Trade: {df_rsi_trades['duracao'].max()}")
    print(f"Taxa de Acerto (Win Rate): {((df_rsi_trades['resultado_liquido_pct'] > 0).sum() / len(df_rsi_trades) * 100):.2f}%")
    print(f"Lucro Médio (nos trades vencedores): {df_rsi_trades[df_rsi_trades['resultado_liquido_pct'] > 0]['resultado_liquido_pct'].mean():.2f}%")
    print(f"Prejuízo Médio (nos trades perdedores): {df_rsi_trades[df_rsi_trades['resultado_liquido_pct'] < 0]['resultado_liquido_pct'].mean():.2f}%")



===== ANÁLISE DE COMPORTAMENTO - ESTRATÉGIA DE TENDÊNCIA =====
Número Total de Trades: 31398
Duração Média de um Trade: 0 days 11:23:07.779476399
Duração Máxima de um Trade: 4 days 22:30:00
Taxa de Acerto (Win Rate): 19.79%
Lucro Médio (nos trades vencedores): 4.13%
Prejuízo Médio (nos trades perdedores): -1.80%


===== ANÁLISE DE COMPORTAMENTO - ESTRATÉGIA RSI =====
Número Total de Trades: 24434
Duração Média de um Trade: 0 days 05:25:30.486207743
Duração Máxima de um Trade: 2 days 10:00:00
Taxa de Acerto (Win Rate): 64.39%
Lucro Médio (nos trades vencedores): 1.37%
Prejuízo Médio (nos trades perdedores): -1.71%
